# What is CSP

We have said that *aiochan* is a CSP-style concurrency library. It turns out we can go a long way towards understanding how to use *aiochan* by just understanding what CSP mean. CSP stands for *Communicating Sequential Processes*. The name CSP first appears in a paper by C.A.R. Hoare in 1978, and some ideas of CSP exist even before that. So CSP is actually quite an old idea.

Let's begin with the last word: *processes*. It seems we have immediately hit upon opportunities for great confusion: python has many packages that have "process" on their names, your operating system has something called "processes" or maybe not, depending on what OS you are using, etc. And they are not what we want to talk about now. *Process* is, unfortunately, one of thoses words in computing that is over-used in many subtly different ways. Here we shall be intentially vague: for us, a *process* is just a group of code that executes fairly independently from the outside world, a group of code that you can mentally think of as a whole entity. The quintessential example is that of a function: a function as a process goes from taking in arguments from the caller, and ends when returning to the caller.

What is *sequential* process, then? If you read the word literally, it means that statements or expressions in the process are executed or evaluated in strict order, from top to bottom. In python this is fairly accurate, if special provision is made for control statements such as `while`, `for` and stuff. A better word might be *deterministic*: even in the presence of control statements, if you know all your variables, it is possible to predict precisely what the next statement or expression to be executed or evaluated is. An example is

In [1]:
x = 10
x += 10
x *= 2
x -= 7
print(x)

33


The above calculates `((10 + 10) * 2) - 7 = 33` and is sequential. If your programming language instead calculates `((10 * 2) + 10) - 7 = 3` then your programming environment has some serious issues. So sequential programs are good, it is what we as humans expect.

However, it is actually very easy to have non-sequential programs. Let's first refactor the above program:

In [2]:
x = 10

def f():
    global x
    print('we start with the value', x)
    x += 10
    x *= 2
    x -= 7
    return x

print(f())

we start with the value 10
33


So far so good. But suppose you have two instances of the `f` process executing concurrently. Then you will have troubles. It is actually possible to illustrate this with python's `threading` module, but for simplicity we will just give an illustration in pseudo code:

```
x = 10

def f1():                  | def f2():
    global x # x == 10     |
                           |     global x # x == 10
    x += 10 # x == 20      |
                           |     x += 10 # x == 30
    x *= 2  # x == 40      |
                           |     x *= 2  # x == 80
    x -= 7  # x == 73      |
    return x               |
                           |     x -= 7  # x == 66
                           |     return x

print('f1 ->', f1())
print('f2 ->', f2())
```

In the above snippet, for the two functions, the order of the execution of statements within the two functions are as listed: from top to down. Note that though within each function itself the sequence is the same as before, when the two functions are taken together statements are interleaved.

We will get the results:

```
f1 -> 73
f2 -> 66
```

Now if you are only in control of `f1` you will be very much baffled. Worse, as you can try for yourself, by tweaking the order further you can get other results. The interleaving makes the execution, and hence the result, non-deterministic. Such processes are *not* considered sequential and not allowed (or, not encouraged) in CSP-style programming.

As you probably can see, the fix is actually not that hard: *don't modify global variables*. Any modifications you do must be local to your process (or in our case, function). You should try and see for yourself that if you follow this advice, then whatever the order of interleaving of the two functions, the results are not affected: they are deterministic and are thus considered *sequential*.

In functional languages, it is sometimes enforced that you cannot make any modifications at all --- any computation you do just returns new values without stepping on the old values. However, in python, this is both unnecessary and unnatural. We only need to disallow operations that can interfere with other processes.

Now, you ask, what disturbed minds would write something like our `f`? Well, be assured that that people who wrote `f` habour no ill intensions. The reason that they reach for global variables is most often because that they want to do some form of input/output (i.e., IO, note that the concept of IO is much broader than file or network accesses): we need to get stuff into our function to compute, and we need to notify others who are also computing what the results of our computations are.

If you think about it, IO is the whole point of computation: *we*, at our keyboards (or touch screens, or whatever your newest VR/AR interaction devices), input something for the computer to compute, the the computer returns the results to *us*. So programs without IO is pretty useless. And using global variables in this case is rather convenient: we just take something (input) that is put inside predetermined boxes (global variables), and when we are done, just put them back. Others, by inspecting the boxes, will know what we have done. By the way, at the lowest level, this is how our current computer architecture dictates. A "pure" function that "returns" something without reference to a box location *is* an illusion. But unfortunately, as we have seen, this crude arrangement results in people stepping on each other and chaos if there are no rules.

In the older mainstream languages, the solution is that we put stickers on the boxes when we want to operate on them: "in use --- don't touch until I'm done!" --- locks, semaphores, etc. This solves the problem, but using locks and similar *concurrency primitives* turn out to be rather delicate and error-prone. A classical example is that you and your friend both want to operate on two boxes A and B. You go forward and put your sticker on A, meanwhile your friend has already put his sticker on B. Now both of you are stuck: unless one of you back off, no one can go forward. Preventing such *deadlocks* means having a whole lot of disciplines and guidelines to follow --- more training to become professional jugglers!

Is there a way out of this? Is there a way to avoid arduous juggler training while still doing concurrency? Yes, and this is what the *Communicating* part of CSP says.

The basic idea is this: when doing computations that must involve read-write IO, we do away with boxes. Instead, we specify *meeting points*, or *points of rendezvous*, for which IO is done. For example, you and your friend both want a book. Instead of putting the book in a box so that both of you can do whatever you want with it whenever you want (and risking the book to be stolen), you just take the book away and do your processing with it. After you are satisfied, you and your friend *meet together* and you *hand off* the book to your friend. Once your friend has the book, she can do anything she wants with it, while you can no longer do anything with it at all. There is no longer any stepping over. If you *really* want your book again, you must arange with your friend for a hand-off again.

Such an arrangment is at least familiar (it is how private properties work). Note that it is principally different from putting stickers on boxes: you just take the book and go off! And the amazing thing is, this arrangement actually solves the majority of concurrency problems. To each *sequential process*, things look as if we are in a non-concurrent program: the only concurrent parts are when we want to do IO, we arrange for rendezvous points. No stickers. No locks. Much less opportunities for deadlocks.

Another very important benefit of communicating over rendezvous points is the respect of privacy, or abstraction barriers. Consider the box-book example again. If we want to use stickers to solve that problem successfully, you and your friend both have to agree on a strategy (for example always start with box A), in other words, you are both opening yourselves up to each other, letting the other know things about how *you* operate, which you may be reluctant to due to various reasons. By contrast, we will find that when using rendezvous points to communicate, often the only thing the other parties need to know is the existence of the rendezvous points. The abstraction barrier is respected!

The rest of this tutorial will go into much more details in how to go about setting up and honouring rendezvous appointments, which in the context of *aiochan*, is called a *channel*, or `Chan`. But first, some environment setups have to be done first, which we will deal with in the next section.

To recap, in the context of CSP (*Communicating Sequantial Processes*):

* Processes are group of codes that can be considered as an independent entity.
* Sequential processes are processes that operate in deterministic order producing deterministic results, without danger of stepping over each other.
* Communicating sequantial processes are sequantial processes that do their (necessary) IO by using rendezvous points only.
* CSP style concurrency enables natural program logic resembling non-concurrent codes, respects abstraction barriers, while at the same time eliminating most dangers of deadlocks and stepping over.